<a href="https://colab.research.google.com/github/AliAI11/fragranceBERT/blob/main/notebooks/gradio_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
# ============================================================================
# fragrance semantic search - gradio demo
# ============================================================================

# install gradio
!pip install gradio sentence-transformers

import gradio as gr
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import torch
import re
from google.colab import files
import zipfile
import os

In [2]:
# ============================================================================
# upload files
# ============================================================================

from google.colab import files
import zipfile
import os

print("upload these 3 files:")
print("1. fragrance-retriever.zip (model)")
print("2. perfume_embeddings.npy")
print("3. perfumes_with_ids.csv")

uploaded = files.upload()

# extract model
os.makedirs('./model', exist_ok=True)
with zipfile.ZipFile('fragrance-retriever.zip', 'r') as zip_ref:
    zip_ref.extractall('./model/')

print("\nfiles uploaded and extracted")

upload these 3 files:
1. fragrance-retriever.zip (model)
2. perfume_embeddings.npy
3. perfumes_with_ids.csv


Saving fragrance-retriever.zip to fragrance-retriever.zip
Saving perfume_embeddings.npy to perfume_embeddings.npy
Saving perfumes_with_ids.csv to perfumes_with_ids.csv

✓ files uploaded and extracted


In [5]:
# ============================================================================
# load model and data
# ============================================================================

print("\nloading model and data...")

# load model
model = SentenceTransformer('./model/')
print(f"model loaded: {model.get_sentence_embedding_dimension()}-dim embeddings")

# load embeddings and data
perfume_embeddings = np.load('perfume_embeddings.npy')
perfumes_df = pd.read_csv('perfumes_with_ids.csv', index_col=0)

print(f"loaded {len(perfumes_df)} perfumes")
print(f"embeddings shape: {perfume_embeddings.shape}")


loading model and data...
model loaded: 384-dim embeddings
loaded 24063 perfumes
embeddings shape: (24063, 384)


In [20]:
# ============================================================================
# search function with fragrantica images and links
# ============================================================================

def search_perfumes(query, top_k=5):
    """
    search for perfumes matching the query
    returns html with images and fragrantica links
    """
    # encode query
    query_embedding = model.encode([query], convert_to_tensor=False)

    # compute similarities
    similarities = cosine_similarity(query_embedding, perfume_embeddings)[0]

    # get top-k indices
    top_indices = np.argsort(similarities)[-top_k:][::-1]

    # build html with cards
    html_parts = ['''
        <style>
            .results-container {
                display: flex;
                flex-wrap: wrap;
                gap: 20px;
                margin: 20px 0;
                justify-content: center;
                padding: 20px;
            }
            .perfume-card {
                width: 200px;
                background: linear-gradient(135deg, #1a0a1a 0%, #2d1a2d 100%);
                border-radius: 16px;
                border: 2px solid #c71585;
                overflow: hidden;
                transition: all 0.3s ease;
                box-shadow: 0 4px 12px rgba(199, 21, 133, 0.3);
            }
            .perfume-card:hover {
                transform: translateY(-8px);
                box-shadow: 0 12px 24px rgba(199, 21, 133, 0.5);
                border-color: #ff69b4;
            }
            .perfume-img {
                width: 100%;
                height: 260px;
                object-fit: cover;
                background: #000000;
                border-bottom: 2px solid #8b008b;
            }
            .perfume-info {
                padding: 15px;
                text-align: center;
            }
            .perfume-name {
                font-size: 14px;
                font-weight: bold;
                color: #ff69b4;
                margin-bottom: 6px;
                text-decoration: none;
                display: block;
                transition: color 0.2s;
            }
            .perfume-name:hover {
                color: #c71585;
            }
            .perfume-brand {
                font-size: 12px;
                color: #9370db;
                margin-bottom: 8px;
                font-weight: 500;
            }
            .perfume-accords {
                font-size: 11px;
                color: #cccccc;
                font-style: italic;
                line-height: 1.4;
            }
            .similarity-badge {
                background: linear-gradient(135deg, #c71585 0%, #8b008b 100%);
                color: white;
                padding: 4px 10px;
                border-radius: 12px;
                font-size: 10px;
                font-weight: bold;
                display: inline-block;
                margin-top: 8px;
            }
            .fragrantica-link {
                display: block;
                margin-top: 8px;
                color: #9370db;
                font-size: 10px;
                text-decoration: none;
                transition: color 0.2s;
            }
            .fragrantica-link:hover {
                color: #ff69b4;
            }
        </style>
        <div class="results-container">
    ''']

    for i, idx in enumerate(top_indices, 1):
        perfume = perfumes_df.iloc[idx]
        perfume_name = perfume["Perfume"]
        brand = perfume["Brand"]
        fragrantica_url = perfume["url"]
        similarity = similarities[idx]

        # extract fragrantica id from url
        # url format: https://www.fragrantica.com/perfume/Brand/Name-12345.html
        match = re.search(r'-(\d+)\.html', fragrantica_url)
        fragrantica_id = match.group(1) if match else idx

        # get accords
        accords = ""
        desc = perfume['description']
        if 'accords:' in desc:
            accords = desc.split('accords:')[1].split('.')[0].strip()

        # fragrantica image url
        img_url = f"https://fimgs.net/mdimg/perfume/375x500.{fragrantica_id}.jpg"

        # create card html
        html_parts.append(f'''
            <div class="perfume-card">
                <a href="{fragrantica_url}" target="_blank">
                    <img src="{img_url}" class="perfume-img"
                         onerror="this.src='https://via.placeholder.com/200x260/1a0a1a/ff69b4?text=No+Image'">
                </a>
                <div class="perfume-info">
                    <a href="{fragrantica_url}" target="_blank" class="perfume-name">
                        {perfume_name}
                    </a>
                    <div class="perfume-brand">{brand}</div>
                    <div class="perfume-accords">{accords}</div>
                    <span class="similarity-badge">Match: {similarity:.1%}</span>
                    <a href="{fragrantica_url}" target="_blank" class="fragrantica-link">
                        View on Fragrantica →
                    </a>
                </div>
            </div>
        ''')

    html_parts.append('</div>')

    return ''.join(html_parts)

In [23]:
# ============================================================================
# gradio interface
# ============================================================================

custom_css = """
.gradio-container {
    background: linear-gradient(135deg, #000000 0%, #1a0a1a 100%) !important;
    font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif !important;
}

#query_box textarea {
    background-color: #1a0a1a !important;
    border: 2px solid #c71585 !important;
    color: white !important;
    font-size: 16px !important;
}

#search_button {
    background: linear-gradient(135deg, #c71585 0%, #8b008b 100%) !important;
    border: none !important;
    color: white !important;
    font-weight: bold !important;
    font-size: 16px !important;
    padding: 12px !important;
    transition: all 0.3s ease !important;
}

#search_button:hover {
    background: linear-gradient(135deg, #ff69b4 0%, #c71585 100%) !important;
    transform: translateY(-2px) !important;
    box-shadow: 0 5px 15px rgba(199, 21, 133, 0.4) !important;
}

#results_box {
    background-color: #000000 !important;
    border: 2px solid #c71585 !important;
    border-radius: 12px !important;
    padding: 10px !important;
}

#tips_box {
    background-color: #1a0a1a !important;
    border: 2px solid #8b008b !important;
    padding: 20px !important;
    border-radius: 12px !important;
}

.markdown-text {
    color: white !important;
}

h1, h2, h3, p, li {
    color: white !important;
}
"""

# example queries
examples = [
    ["warm vanilla for cozy winter evenings"],
    ["fresh citrus for spring mornings"],
    ["romantic floral for date night"],
    ["confident masculine woody leather"],
    ["energizing coffee and bergamot"],
    ["calming lavender for bedtime"],
    ["sweet gourmand with caramel"],
    ["elegant powdery iris and musk"]
]

# create interface
with gr.Blocks(css=custom_css, theme=gr.themes.Base()) as demo:

    gr.Markdown(
        """
        # Semantic Fragrance Search
        ### Find your perfect perfume using natural language :)

        Describe the scent you're looking for using emotions, seasons, occasions, or specific notes.
        The AI will find perfumes that match your description from the Fragrantica database.
        """,
        elem_classes="markdown-text"
    )

    with gr.Row():
        with gr.Column(scale=2):
            query_input = gr.Textbox(
                label="Describe your ideal fragrance",
                placeholder="e.g., 'warm vanilla for cozy winter evenings' or 'fresh citrus for spring mornings'",
                lines=3,
                elem_id="query_box"
            )

            with gr.Row():
                top_k = gr.Slider(
                    minimum=3,
                    maximum=10,
                    value=5,
                    step=1,
                    label="Number of results",
                    info="How many perfumes to show"
                )

            search_btn = gr.Button(
                "Search Fragrances",
                variant="primary",
                elem_id="search_button"
            )

        with gr.Column(scale=1):
            gr.Markdown(
                """
                ### Tips
                - Use descriptive language: *warm, fresh, elegant, playful*
                - Mention occasions: *date night, office, gym*
                - Include seasons: *winter, spring, summer, autumn*
                - Describe feelings: *romantic, confident, calming*
                - Name specific notes: *vanilla, citrus, woody, floral*

                ### About
                - **24,063** fragrances from Fragrantica
                - Click any perfume for full details
                - Images and info from Fragrantica.com
                """,
                elem_id="tips_box",
                elem_classes="markdown-text"
            )

    # results (html output with images)
    results_output = gr.HTML(
        label="Search Results",
        elem_id="results_box"
    )

    # example queries
    gr.Markdown(
        "### Example Queries (click to try)",
        elem_classes="markdown-text"
    )
    gr.Examples(
        examples=examples,
        inputs=query_input,
        label=None
    )

    # footer
    gr.Markdown(
        """
        ---
        **Model**: Sentence-BERT fine-tuned on 24,063 fragrances from Fragrantica
        **Technique**: Bi-encoder architecture with contrastive learning
        **Dataset**: Synthetic queries generated using LLM (Qwen2.5-7B)

        *Built as part of VT CS 5804 Machine Learning project*

        **Data Source**: [Fragrantica.com](https://www.fragrantica.com) - The world's largest perfume database
        """,
        elem_classes="markdown-text"
    )

    # connect button to search function
    search_btn.click(
        fn=search_perfumes,
        inputs=[query_input, top_k],
        outputs=results_output
    )

    # also allow enter key
    query_input.submit(
        fn=search_perfumes,
        inputs=[query_input, top_k],
        outputs=results_output
    )

/tmp/ipython-input-3599352132.py:70: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(css=custom_css, theme=gr.themes.Base()) as demo:
/tmp/ipython-input-3599352132.py:70: DeprecationWarning: The 'css' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'css' to Blocks.launch() instead.
  with gr.Blocks(css=custom_css, theme=gr.themes.Base()) as demo:


In [ ]:
# ============================================================================
# launch the demo
# ============================================================================

print("\n" + "="*80)
print("launching gradio demo...")
print("="*80)

# launch with share=true to get public link
demo.launch(
    share=True,  # creates public link
    debug=True,
    show_error=True
)


launching gradio demo...
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://539afd02c2b45d4165.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
